In [ ]:
import pypsa
import pandas as pd
import geopandas as gpd
import sys

In [ ]:

sys.path.append("../processed_spatial_res")

from get_spatial_res import get_spatial_res

regions = get_spatial_res()
print(regions)

                                                                      geometry  \
region_5                                                                         
Kärnten/Steiermark           POLYGON ((14.43537 46.44343, 14.43537 46.44345...   
Niederösterreich/Burgenland  POLYGON ((16.06715 46.84889, 16.0672 46.84884,...   
Oberösterreich/Salzburg      POLYGON ((13.69947 47.00479, 13.69858 47.00534...   
Tirol/Vorarlberg             MULTIPOLYGON (((12.39612 46.71143, 12.39598 46...   
Wien                         POLYGON ((16.29905 48.12999, 16.29897 48.12999...   

                            GID_0  COUNTRY          NAME_1  \
region_5                                                     
Kärnten/Steiermark            AUT  Austria         Kärnten   
Niederösterreich/Burgenland   AUT  Austria      Burgenland   
Oberösterreich/Salzburg       AUT  Austria  Oberösterreich   
Tirol/Vorarlberg              AUT  Austria           Tirol   
Wien                          AUT  Austria           

In [ ]:
sys.path.append("../processed_capacity_factors") ### dieser Schritt muss für diesen Folder auch passieren ###
sys.path.append("../processed_existing_conv_pp")

import importlib
import cutout_local
importlib.reload(cutout_local)

import get_existing_conv_pp
importlib.reload(get_existing_conv_pp)

<module 'get_existing_conv_pp' from 'c:\\Users\\steli\\DSESM\\data\\pypsa_model\\../processed_existing_conv_pp\\get_existing_conv_pp.py'>

In [ ]:
sys.path.append("../processed_existing_conv_pp")
from get_existing_conv_pp import (get_existing_conv_pp)

In [ ]:
gdf_pp = get_existing_conv_pp()

In [ ]:
regions_3035 = regions.to_crs(3035)
pp_3035 = gdf_pp.to_crs(3035)

In [ ]:
pp_with_region["region"] = pp_with_region["region_5"]

In [ ]:
conv_cap = (
    pp_with_region
    .groupby(["region", "primary_fuel"])["capacity_mw"]
    .sum()
)
conv_cap = conv_cap.loc[
    conv_cap.index.get_level_values("primary_fuel")
    .isin(["Gas", "Hydro"])
]
conv_cap

In [ ]:
for c in ["gas", "hydro"]:
    if c not in n.carriers.index:
        n.add("Carrier", c)

In [ ]:
for (region, fuel), cap in conv_cap.items():
    if fuel != "Gas":
        continue

    n.add(
        "Generator",
        name=f"gas_existing_{region}",
        bus=region,
        carrier="gas",
        p_nom=cap,
        p_nom_extendable=False,
        efficiency=0.5, #default
        marginal_cost=80.0 #default
    )

In [ ]:
for (region, fuel), cap in conv_cap.items():
    if fuel != "Hydro":
        continue

    n.add(
        "Generator",
        name=f"hydro_existing_{region}",
        bus=region,
        carrier="hydro",
        p_nom=cavp,
        p_nom_extendable=False,
        p_max_pu=0.45, #default
        marginal_cost=5.0 #default 
    )

In [ ]:
sys.path.append("../processed_capacity_factors")
from cutout_local import (
    get_cutout,
    get_availability,
    get_pv_and_wind,
    get_excluders
)

In [ ]:
excluder_solar, excluder_wind = get_excluders()

In [ ]:
# 4) Availability-Matrizen
A_pv, A_w = get_availability(
    cutout=cutout,
    regions=regions,
    excluder_solar=excluder_solar,
    excluder_wind=excluder_wind
)

In [ ]:
# 5) PV & Wind Potenziale + CFs
pv_cap, pv_cf, wind_cap, wind_cf = get_pv_and_wind(
    cutout=cutout,
    regions=regions,
    A_pv=A_pv,
    A_w=A_w
)
pv_cf_df = pv_cf.to_pandas()

In [ ]:
n=pypsa.Network()
regions=get_spatial_res()
regions

In [ ]:
for region, row in regions.iterrows():
    n.add("Bus",name=region,x=row["REP_POINT"].x,y=row["REP_POINT"].y,carrier="electricity")

n.buses #check

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6,4))
regions.plot(ax=ax, color="lightgrey", edgecolor="black")
ax.scatter(n.buses.x, n.buses.y, color="red", zorder=5)
ax.set_title("PyPSA-Buses (5 Regionen)")
plt.show()

In [ ]:
n.set_snapshots(pv_cf_df.index)

In [ ]:
dummy_load = pd.Series(100.0,index=n.snapshots)
for region in regions.index:
    n.add("Load",name=f"load_{region}",bus=region,p_set=dummy_load)

n.loads
n.loads_t.p_set.head()

In [ ]:
pv_cf_df = pv_cf.to_pandas()
pv_cap_s = pv_cap.to_pandas()
for region in regions.index:
    n.add(
        "Generator",
        name=f"PV_{region}",
        bus=region,
        carrier="solar",
        p_nom_extendable=True,
        p_nom_max=pv_cap_s.loc[region],
        p_max_pu=pv_cf_df[region],
        marginal_cost=0.0,
        capital_cost=1.0
    )

In [ ]:
wind_cf_df = wind_cf.to_pandas()
wind_cap_s = wind_cap.to_pandas()

for region in regions.index:
    n.add(
        "Generator",
        name=f"Wind_{region}",
        bus=region,
        carrier="wind",
        p_nom_extendable=True,
        p_nom_max=wind_cap_s.loc[region],
        p_max_pu=wind_cf_df[region],
        marginal_cost=0.0,
        capital_cost=1.0
    )

In [ ]:
n.generators.head()
n.generators_t.p_max_pu.head()

In [ ]:
neighbours = [
    ("Wien", "Niederösterreich/Burgenland"),
    ("Niederösterreich/Burgenland", "Oberösterreich/Salzburg"),
    ("Oberösterreich/Salzburg", "Tirol/Vorarlberg"),
    ("Oberösterreich/Salzburg", "Kärnten/Steiermark"),
]

rep_points = regions["REP_POINT"].to_crs(3035)
def distance_km(r1, r2):
    d = rep_points.loc[r1].distance(rep_points.loc[r2])
    return 1.5 * d / 1000  

In [ ]:
COST_PER_MW_KM = 700  
for r1, r2 in neighbours:
    length_km = distance_km(r1, r2)
    capital_cost = COST_PER_MW_KM * length_km

    n.add(
        "Link",
        name=f"line_{r1}_{r2}",
        bus0=r1,
        bus1=r2,
        p_nom_extendable=True,
        p_min_pu=-1,      # bidirektional
        p_max_pu=1,
        efficiency=1.0,   # keine Verluste
        capital_cost=capital_cost
    )

In [ ]:
n.links.capital_cost